In [1]:
import findspark

In [2]:
findspark.init('/home/ubuntu/spark-3.0.0-bin-hadoop2.7')

In [3]:
import pyspark

In [4]:
from pyspark.sql import SparkSession

In [5]:
spark = SparkSession.builder.appName('ops').getOrCreate()

22/05/14 18:38:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [6]:
df = spark.read.csv('appl_stock.csv', inferSchema=True, header=True)

In [7]:
df.printSchema()

root
 |-- Date: string (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



In [8]:
df.describe().show()

+-------+----------+------------------+------------------+------------------+-----------------+-------------------+------------------+
|summary|      Date|              Open|              High|               Low|            Close|             Volume|         Adj Close|
+-------+----------+------------------+------------------+------------------+-----------------+-------------------+------------------+
|  count|      1762|              1762|              1762|              1762|             1762|               1762|              1762|
|   mean|      null| 313.0763111589103| 315.9112880164581| 309.8282405079457|312.9270656379113|9.422577587968218E7| 75.00174115607275|
| stddev|      null|185.29946803981522|186.89817686485767|183.38391664371008|185.1471036170943|6.020518776592709E7| 28.57492972179906|
|    min|2010-01-04|              90.0|         90.699997|         89.470001|        90.279999|           11475900|         24.881912|
|    max|2016-12-30|        702.409988|        705.0700

In [9]:
df.head(3)

[Row(Date='2010-01-04', Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039),
 Row(Date='2010-01-05', Open=214.599998, High=215.589994, Low=213.249994, Close=214.379993, Volume=150476200, Adj Close=27.774976000000002),
 Row(Date='2010-01-06', Open=214.379993, High=215.23, Low=210.750004, Close=210.969995, Volume=138040000, Adj Close=27.333178000000004)]

In [10]:
df.createOrReplaceTempView('stock')

In [14]:
query = spark.sql('SELECT DATE, OPEN, VOLUME FROM STOCK WHERE VOLUME >= 311488100')

In [15]:
query.show()

+----------+------------------+---------+
|      DATE|              OPEN|   VOLUME|
+----------+------------------+---------+
|2010-01-26|205.95000100000001|466777500|
|2010-01-27|        206.849995|430642100|
|2010-01-29|        201.079996|311488100|
|2010-05-06|        253.830002|321465200|
|2010-05-07|243.71001099999998|419004600|
|2010-05-20|241.88000899999997|320728800|
|2011-01-18|        329.520012|470249500|
|2012-02-15|        514.259995|376530000|
|2012-03-14|        578.050026|354711000|
|2012-11-16|        525.200005|316723400|
|2013-01-24|        460.000008|365213100|
+----------+------------------+---------+



In [16]:
df.select(['date', 'open', 'volume']).filter(df['volume'] > 311488100).show()

+----------+------------------+---------+
|      date|              open|   volume|
+----------+------------------+---------+
|2010-01-26|205.95000100000001|466777500|
|2010-01-27|        206.849995|430642100|
|2010-05-06|        253.830002|321465200|
|2010-05-07|243.71001099999998|419004600|
|2010-05-20|241.88000899999997|320728800|
|2011-01-18|        329.520012|470249500|
|2012-02-15|        514.259995|376530000|
|2012-03-14|        578.050026|354711000|
|2012-11-16|        525.200005|316723400|
|2013-01-24|        460.000008|365213100|
+----------+------------------+---------+



In [13]:
query2 = spark.sql('SELECT OPEN, CLOSE FROM STOCK WHERE OPEN > 200 AND CLOSE < 200')

In [14]:
query2.show()

+------------------+----------+
|              OPEN|     CLOSE|
+------------------+----------+
|206.78000600000001|    197.75|
|        204.930004|199.289995|
|        201.079996|192.060003|
+------------------+----------+



In [23]:
df.select(['open', 'close']).filter((df['open'] > 200) & ~(df['close'] < df['open'])).show()

+------------------+------------------+
|              open|             close|
+------------------+------------------+
|        213.429998|        214.009998|
|        210.299994|211.98000499999998|
|        207.870005|        210.650002|
|        208.330002|        215.039995|
|202.51000200000001|        203.070002|
|        206.849995|        207.880005|
|        201.940002|        203.399996|
|        201.629995|        202.929998|
|        202.379999|204.61999699999998|
|        205.749996|        208.990004|
|        208.940002|        209.329998|
|        209.279997|210.71000299999997|
|        214.940006|218.95000499999998|
|218.31000299999997|        223.020004|
|        223.829996|224.83999300000002|
|        223.909998|        225.500008|
|        224.180004|224.44999700000002|
|        224.100002|        224.650002|
|220.46999900000003|        224.750004|
|        225.640011|        228.359993|
+------------------+------------------+
only showing top 20 rows



# .collect() to fetch a list of the row >>> dictionary

In [30]:
result = df.filter(df['Low'] > 197.16).collect()

In [31]:
print(result)

[Row(Date='2010-01-04', Open=213.429998, High=214.499996, Low=212.38000099999996, Close=214.009998, Volume=123432400, Adj Close=27.727039), Row(Date='2010-01-05', Open=214.599998, High=215.589994, Low=213.249994, Close=214.379993, Volume=150476200, Adj Close=27.774976000000002), Row(Date='2010-01-06', Open=214.379993, High=215.23, Low=210.750004, Close=210.969995, Volume=138040000, Adj Close=27.333178000000004), Row(Date='2010-01-07', Open=211.75, High=212.000006, Low=209.050005, Close=210.58, Volume=119282800, Adj Close=27.28265), Row(Date='2010-01-08', Open=210.299994, High=212.000006, Low=209.06000500000002, Close=211.98000499999998, Volume=111902700, Adj Close=27.464034), Row(Date='2010-01-11', Open=212.79999700000002, High=213.000002, Low=208.450005, Close=210.11000299999998, Volume=115557400, Adj Close=27.221758), Row(Date='2010-01-12', Open=209.18999499999998, High=209.76999500000002, Low=206.419998, Close=207.720001, Volume=148614900, Adj Close=26.91211), Row(Date='2010-01-13',

In [32]:
print(result[1])

Row(Date='2010-01-05', Open=214.599998, High=215.589994, Low=213.249994, Close=214.379993, Volume=150476200, Adj Close=27.774976000000002)


In [33]:
row = result[0]

In [34]:
row.asDict()

{'Date': '2010-01-04',
 'Open': 213.429998,
 'High': 214.499996,
 'Low': 212.38000099999996,
 'Close': 214.009998,
 'Volume': 123432400,
 'Adj Close': 27.727039}

In [35]:
row.asDict()['Low']

212.38000099999996

# Group by Aggregation